In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split

C:\Users\Lidia\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [3]:
train_no_id = train.drop('id',axis=1)
train_no_id = train_no_id[['target','ps_ind_01',
'ps_ind_03',
'ps_ind_05_cat',
'ps_ind_07_bin',
'ps_ind_15',
'ps_ind_16_bin',
'ps_reg_01',
'ps_reg_02',
'ps_reg_03',
'ps_car_01_cat',
'ps_car_03_cat',
'ps_car_07_cat',
'ps_car_12',
'ps_car_13',
'ps_car_14',
'ps_car_15']]
train70, train30 = train_test_split(train_no_id,test_size=0.3,random_state=192)

In [8]:
with tf.name_scope('logres'):
    x = tf.placeholder(tf.float32, [None, 16])
    W = tf.Variable(tf.zeros([16, 1]))
    b = tf.Variable(tf.zeros([1]))
    y = tf.matmul(x, W)+b
    y_ = tf.placeholder(tf.float32, [None, 1])

In [20]:
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y_, logits=y))
    train_step = tf.train.AdamOptimizer(0.05).minimize(cross_entropy)

In [21]:
with tf.name_scope('accuracy'):
    accuracy_op = tf.metrics.auc(y_,tf.sigmoid(y))

In [24]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for j in range(50):
        for i in range(100):
            train_batch = train70.sample(frac=0.0001)
            batch_xs = train_batch.drop('target',axis=1)
            batch_ys = train_batch['target'].values.reshape(train_batch.shape[0],1)
            sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
        sess.run(tf.local_variables_initializer())
        print(sess.run(accuracy_op,feed_dict={x: train70.drop('target',axis=1), y_: train70['target'].values.reshape(train70.shape[0],1)})) 
    sess.run(tf.local_variables_initializer())
    print(sess.run(accuracy_op,feed_dict={x: train30.drop('target',axis=1), y_: train30['target'].values.reshape(train30.shape[0],1)}))     

(0.0, 0.56789261)
(0.0, 0.55299842)
(0.0, 0.52935439)
(0.0, 0.57008266)
(0.0, 0.54601264)
(0.0, 0.54097188)
(0.0, 0.57708937)


KeyboardInterrupt: 